In [8]:
import numpy as np

def wh_ratio_narrow(Z0, eps_r):
    """
    w/h för smala ledare (w/h < 2)
    Z0 : karakteristisk impedans [ohm]
    eps_r : relativ permittivitet
    """
    A = (Z0 / 120.0) * np.sqrt(2 * (eps_r + 1.0)) + (eps_r - 1.0) / (eps_r + 1.0) * (0.226 + 0.121 / eps_r)
    # Hammerstad-inversion
    wh = 4 * ((np.exp(A)/2) - np.exp(-A))**-1
    
    return wh


def wh_ratio_wide(Z0, eps_r):
    """
    w/h för bredare ledare (w/h > 2)
    Z0 : karakteristisk impedans [ohm]
    eps_r : relativ permittivitet
    """
    # B ≈ 60 * pi^2 / (Z0 * sqrt(eps_r))  (≈ 377*pi / (2*Z0*sqrt(eps_r)))
    B = 60.0 * np.pi**2 / (Z0 * np.sqrt(eps_r))

    term1 = (eps_r - 1.0) / (np.pi * eps_r) * (np.log(B - 1.0) + 0.293 - 0.517 / eps_r)

    term2 = (2.0 / np.pi) * (B - 1.0 - np.log(2.0 * B - 1.0))

    wh =  (term1 + term2)
    return wh


def wh_ratio(Z0, eps_r):
    """
    Wrapper: beräkna w/h och välj rätt formel.
    Kör först 'narrow'-formeln; om resultatet blir >2
    räknar vi om med 'wide'-formeln.
    """
    wh = wh_ratio_narrow(Z0, eps_r)
    if wh > 2.0:
        #print("wh was wide: ", wh)
        wh = wh_ratio_wide(Z0, eps_r)
    return wh


def eps_eff(eps_r, wh_ratio):
    """
    Effektiv permittivitet (formeln i lab-PDF:en):
        eps_eff = (eps_r+1)/2 + (eps_r-1)/2 * 1/sqrt(1 + 12*h/w)
    h, w i samma längdenhet (t.ex. mm)
    """
    return (eps_r + 1.0)/2.0 + (eps_r - 1.0)/2.0 * 1.0 / np.sqrt(1.0 + 12.0 * wh_ratio**-1)


In [ ]:
Z0 = 50      # ohm
er = 2.4     # DiClad 527
C = 4
S_31 = 10**(-C/20)
freq = 4e9

Y_c = 1 / Z0

Y_1 = Y_c / (np.sqrt(1-S_31**2))
Z_1 = 1 / Y_1

Y_2 = S_31 * Y_1
Z_2 = 1 / Y_2

wh_ratio_Z_0 = wh_ratio(Z0, er)
wh_ratio_Z_1 = wh_ratio(Z_1, er)
wh_ratio_Z_2 = wh_ratio(Z_2, er)

width_Z_0 = wh_ratio_Z_0 * 0.78
width_Z_1 = wh_ratio_Z_1 * 0.78
width_Z_2 = wh_ratio_Z_2 * 0.78



print(f"S21 =", Y_c/Y_1)
print(f"S31 =", S_31)
print(f" ")

print(f"Y1 =", Y_1)
print(f"Z1 =", Z_1)
print(f"Y2 =", Y_2)
print(f"Z2 =", Z_2)
print(f" ")

print(f"w/h at Z_0: {Z0:.0f} ohms = {wh_ratio_Z_0:.4f} | w = {width_Z_0:.4f} mm")
print(f"w/h at Z_1: {Z_1:.4f} ohms = {wh_ratio_Z_1:.4f} | w = {width_Z_1:.4f} mm")
print(f"w/h at Z_2: {Z_2:.4f} ohms = {wh_ratio_Z_2:.4f} | w = {width_Z_2:.4f} mm")

S21 = 0.7758175232917227
S31 = 0.6309573444801932
 
Y1 = 0.025779257879020096
Z1 = 38.79087616458614
Y2 = 0.01626561209401662
Z2 = 61.479395562854634
 
w/h at Z_0: 50 ohms = 2.9032 | w = 2.2645 mm
w/h at Z_1: 38.7909 ohms = 4.1914 | w = 3.2693 mm
w/h at Z_2: 61.4794 ohms = 2.0915 | w = 1.6314 mm


In [37]:
eps_eff_z1 = eps_eff(er, wh_ratio_Z_1)
eps_eff_z2 = eps_eff(er, wh_ratio_Z_2)

quarter_lambda_Z1 = 299758292 / (4 * freq * np.sqrt(eps_eff_z1))
quarter_lambda_Z2 = 299758292 / (4 * freq * np.sqrt(eps_eff_z2))


print(f"effective eps Z1 = {eps_eff_z1:.4f}")
print(f"effective eps Z2 = {eps_eff_z2:.4f}")

print(f"quarter wavelength for substrate Z1 = {quarter_lambda_Z1 * 1000:.4f} mm")
print(f"quarter wavelength for substrate Z2 = {quarter_lambda_Z2 * 1000:.4f} mm")



effective eps Z1 = 2.0562
effective eps Z2 = 1.9697
quarter wavelength for substrate Z1 = 13.0654 mm
quarter wavelength for substrate Z2 = 13.3491 mm
